In [1]:
import math
import random, sys

Некоторые реализации вспомогательных функций, например rational_to_contfrac(x,y), а так же части алгоритма Винера были взяты отсюда:

https://github.com/pablocelayes/rsa-wiener-attack

Идея реализации p-метода Полларда была взята из статьи https://cyberleninka.ru/article/n/realizatsiya-metoda-faktorizatsii-pollarda-na-yazyke-c

Вспомогательные функции

In [2]:
def egcd(a,b):
    '''
    Расширенный алгоритм Евклида
    Возвращает x, y, gcd(a,b) такие, что ax + by = gcd(a,b)
    
    Extended Euclidean Algorithm
    returns x, y, gcd(a,b) such that ax + by = gcd(a,b)
    '''
    u, u1 = 1, 0
    v, v1 = 0, 1
    while b:
        q = a // b
        u, u1 = u1, u - q * u1
        v, v1 = v1, v - q * v1
        a, b = b, a - q * b
    return u, v, a

def gcd(a,b):

    a,b = (b,a) if a<b else (a,b)
    
    while b:
        a,b=b,a%b
    return a

def modInverse(e,n):
    '''
    Возвращает d: de = 1 (mod n)
    e должно быть взаимно просто с n
    
    d such that de = 1 (mod n)
    e must be coprime to n
    '''
    return egcd(e,n)[0]%n

def totient(p,q):
    '''
    Calculates the totient of pq
    '''
    return (p-1)*(q-1)

def bitlength(x):
    '''
    Calculates the bitlength of x
    '''
    assert x >= 0
    n = 0
    while x > 0:
        n = n+1
        x = x>>1
    return n


def isqrt(n):
    '''
    Calculates the integer square root
    for arbitrary large nonnegative integers
    '''
    if n < 0:
        raise ValueError('square root is not defined for negative numbers')
    
    if n == 0:
        return 0
    a, b = divmod(bitlength(n), 2)
    x = 2**(a+b)
    while True:
        y = (x + n//x)//2
        if y >= x:
            return x
        x = y


def is_perfect_square(n):
    '''
    If n is a perfect square it returns sqrt(n),
    
    otherwise returns -1
    '''
    h = n & 0xF; #last hexadecimal "digit"
    
    if h > 9:
        return -1 # return immediately in 6 cases out of 16.

    # Take advantage of Boolean short-circuit evaluation
    if ( h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8 ):
        # take square root if you must
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1

In [3]:
def test_is_perfect_square():

    testsuit = [4, 0, 15, 25, 18, 901, 1000, 1024]
    
    for n in testsuit:
        print("Is ", n, " a perfect square?")
        if is_perfect_square(n)!= -1:
            print("Yes!")
        else:
            print("Nope")

In [4]:
test_is_perfect_square()

Testing is_perfect_square
Is  4  a perfect square?
Yes!
Is  0  a perfect square?
Yes!
Is  15  a perfect square?
Nope
Is  25  a perfect square?
Yes!
Is  18  a perfect square?
Nope
Is  901  a perfect square?
Nope
Is  1000  a perfect square?
Nope
Is  1024  a perfect square?
Yes!


In [5]:
def rational_to_contfrac(x,y):
    '''
    Converts a rational x/y fraction into
    a list of partial quotients [a0, ..., an]
    '''
    a = x//y
    pquotients = [a]
    while a * y != x:
        x,y = y,x-a*y
        a = x//y
        pquotients.append(a)
    return pquotients


def convergents_from_contfrac(frac):
    '''
    computes the list of convergents
    using the list of partial quotients
    '''
    convs = [];
    for i in range(len(frac)):
        convs.append(contfrac_to_rational(frac[0:i]))
    return convs


def contfrac_to_rational (frac):
    '''Converts a finite continued fraction [a0, ..., an]
     to an x/y rational.
     '''
    if len(frac) == 0:
        return (0,1)
    num = frac[-1]
    denom = 1
    for _ in range(-2,-len(frac)-1,-1):
        num, denom = frac[_]*num+denom, num
    return (num,denom)

In [6]:
def test1():
    '''
    Verify that the basic continued-fraction manipulation stuff works.
    '''
    testnums = [(1, 1), (1, 2), (5, 15), (27, 73), (73, 27)]
    for r in testnums:
        (num, denom) = r
        print('rational number:')
        print(r)

        contfrac = rational_to_contfrac (num, denom)
        print('continued fraction:')
        print(contfrac)

        print('convergents:')
        print(convergents_from_contfrac(contfrac))
        print('***********************************')

In [7]:
test1()

rational number:
(1, 1)
continued fraction:
[1]
convergents:
[(0, 1)]
***********************************
rational number:
(1, 2)
continued fraction:
[0, 2]
convergents:
[(0, 1), (0, 1)]
***********************************
rational number:
(5, 15)
continued fraction:
[0, 3]
convergents:
[(0, 1), (0, 1)]
***********************************
rational number:
(27, 73)
continued fraction:
[0, 2, 1, 2, 2, 1, 2]
convergents:
[(0, 1), (0, 1), (1, 2), (1, 3), (3, 8), (7, 19), (10, 27)]
***********************************
rational number:
(73, 27)
continued fraction:
[2, 1, 2, 2, 1, 2]
convergents:
[(0, 1), (2, 1), (3, 1), (8, 3), (19, 7), (27, 10)]
***********************************


Генерация простых чисел при помощи алгоритма Миллера-Рабина

In [8]:
def miller_rabin_pass(a, s, d, n):
    ''' 
    n is an odd number with
    n-1 = (2^s)d, and d odd
    and a is the base: 1 < a < n-1

    returns True if n passes the MillerRabinTest for a 
    '''
    a_to_power = pow(a, d, n)
    i=0
    #Invariant: a_to_power = a^(d*2^i) mod n
    
    # we test whether (a^d) = 1 mod n
    if a_to_power == 1:
        return True

    # we test whether a^(d*2^i) = n-1 mod for 0<=i<=s-1
    while(i < s-1):
        if a_to_power == n - 1:
            return True
        a_to_power = (a_to_power * a_to_power) % n
        i+=1

    # we reach here if the test failed until i=s-2
    return a_to_power == n - 1


def miller_rabin(n):
    '''
    Applies the MillerRabin Test to n (odd)

    returns True iff n passes the MillerRabinTest for K random bases
    '''
    #Compute s and d such that n-1 = (2^s)d, with d odd
    d = n-1
    s = 0
    while d%2 == 0:
        d >>= 1
        s+=1

    #Applies the test K times
    #The probability of a false positive is less than (1/4)^K
    K = 20

    i=1
    while(i<=K):
    # 1 < a < n-1
        a = random.randrange(2,n-1)
        if not miller_rabin_pass(a, s, d, n):
            return False
        i += 1

    return True


def gen_prime(nbits):
    '''
    Generates a prime of b bits using the
    miller_rabin_test
    '''
    while True:
        p = random.getrandbits(nbits)
        #force p to have nbits and be odd
        p |= 2**nbits | 1
        if miller_rabin(p):
            return p
            break

def gen_prime_range(start, stop):
    '''
    Generates a prime within the given range
    using the miller_rabin_test
    '''
    while True:
        p = random.randrange(start,stop-1)
        p |= 1
        if miller_rabin(p):
            return p
            break

In [9]:
def test_MR(name, n):
    
    if name == "test":
        print (miller_rabin(n) and "PRIME" or "COMPOSITE")
    elif name == "genprime":
        nbits = int(n)
        print(gen_prime(nbits))

In [10]:
test_MR('test', 1669)

PRIME


In [12]:
test_MR('test', 2002)

COMPOSITE


In [11]:
test_MR('genprime', 1933)

1485378020270680624625363557430696679376946752204920726593074627036058520796480339603575122958022816417941424453102689014201227445918007516903159419521092910169466505898022558661603640128512594763003730081764307803776715008870826706213220321390615973595625777904595609527276884692029820892334511566467986467865675663732840826595471696042680819256115652509614426244020155646405907917083835363744701572944653590479679186702523908099941319049541903350246272717214293452570398150777636383656695241079483080449400622573702698669141780732355228549045426532917108583584833399761994956103697


Генерация ключей

In [34]:
def getPrimePair(bits=512):

    assert bits%4==0
    
    p = gen_prime(bits)
    q = gen_prime_range(p+1, 2*p)
    
    return p,q

def generateKeys(nbits=1024):
    '''
    Generates a key pair
        public = (e,n)
        private = d 
    such that
        n is nbits long
    '''
    # nbits >= 1024 is recommended
    assert nbits%4==0
    
    p,q = getPrimePair(nbits//2)
    n = p*q
    phi = totient(p, q)
        
    # generate a d such that:
    #     (d,n) = 1
    good_d = False
    while not good_d:
        d = random.getrandbits(nbits//4)
        if (gcd(d,phi) == 1):
            good_d = True
                    
    e = modInverse(d,phi)
    return e,n,d

In [37]:
def testkeys():
    
    for i in range(5):
        e,n,d = generateKeys()
        print ("Public key:")
        print("e =")
        print(e)
        print("n =")
        print(n)
        print ("Private key:")
        print("d =")
        print(d)
        print("-----------------------")

In [38]:
testkeys()

Public key:
e =
295194298337628501400653233653165369178603099381624520730714550020639300776780028649521638928268552539677526769077583715860995599045381413359918427744911386533943047269095568786516935085871624189640862063476481735185237949041784244304492196669467322742098182791126998665366427038272002500113178446203896889553
n =
1273728618462068819198768588537171353485617033157968977013455260145085699072797552810495895105390287831586321604155776977300932909060787209240776451314371835342769252885267107251070086478315226863015872166430312483292227872126368396126915885787160335954208239387877105267550374525729865792368287302611494430333
Private key:
d =
59176897491292277193133700296376371239087836714728285272643217028237134730689
-----------------------
Public key:
e =
30422480991548369080991321542289809481713392671586979944994792477012161072047936683442734536591151482029007264327524819598996353823376434411176030681070266910342665635426686526616414347185870919508938906872475441929026509291

### Реализация атак

Атака факторизации модуля p-методом Полларда

In [26]:
def p_Pollard(n : int, x : int, y : int, stage : int, i : int): 
    
    if math.gcd(n, abs(x - y)) == 1:
        if i == stage:
            return p_Pollard(n, (x*x+1)%n, x, stage*2, i+1)
#         print(p_Pollard(n, (x*x+1)%n, y, stage, i+1))
        return p_Pollard(n, (x*x+1)%n, y, stage, i+1)
    else:
#         print(math.gcd(n, abs(x-y)))
        return math.gcd(n, abs(x-y))

In [23]:
def test_pPollard(N):
    
    n = N
    x = random.randint(1, N-2)
    y = 1
    i = 0
    stage = 2
    
    p_Pollard(n, x, y, stage, i)

In [27]:
test_pPollard(39)

3
3
3


Атака факторизации модуля (p-1)-методом Полларда. 

In [ ]:
Простые числа p и q в системе RSA
необходимо также выбирать, исходя из тех соображений, чтобы p±1, q±1 имели по крайней
мере один простой делитель, больший 1020, в противном случае p можно эффективно найти,
используя (p-1)-алгоритм Полларда.
Пусть N > 1 составное число. Следующий алгоритм с некоторой вероятностью
возвращает нетривиальный делитель N.
1. Случайно выбрать а Є Zn. Выбрать положительное целое k=НОК(1,2, …, B),
для соответствующей границы B.
2. Вычислить ak≡a
k
(mod N).
3. Вычислить f=НОД(ak-1,N).
4. Если 1<f<N, то f – делитель N, вывести f и перейти к шагу 6.
5. Иначе перейти к шагу 2 и выбрать новое a и k.
6. Завершить алгоритм.


In [ ]:
template <class T>
T pollard_p_1 (T n)
{
	// параметры алгоритма, существенно влияют на производительность и качество поиска
	const T b = 13;
	const T q[] = { 2, 3, 5, 7, 11, 13 };

	// несколько попыток алгоритма
	T a = 5 % n;
	for (int j=0; j<10; j++)
	{

		// ищем такое a, которое взаимно просто с n
		while (gcd (a, n) != 1)
		{
			mulmod (a, a, n);
			a += 3;
			a %= n;
		}

		// вычисляем a^M
		for (size_t i = 0; i < sizeof q / sizeof q[0]; i++)
		{
			T qq = q[i];
			T e = (T) floor (log ((double)b) / log ((double)qq));
			T aa = powmod (a, powmod (qq, e, n), n);
			if (aa == 0)
				continue;
			
			// проверяем, не найден ли ответ
			T g = gcd (aa-1, n);
			if (1 < g && g < n)
				return g;
		}

	}

	// если ничего не нашли
	return 1;

}

In [ ]:
def lcm(a, b):
    return (a * b) // math.gcd(a, b)

def point2()

def pmin1_Pollard():
    
    k = 1
    for i in range(1,B):
        k = lcm(k,i)
    
    a_k = 
    
    f = math.gcd(a_k-1, N)
    
    if f>1 and f<N:
        return f
    else:
        point2() # ?
    

    

Атака Винера

In [28]:
def hack_RSA(e,n):
    '''
    Finds d knowing (e,n)
    applying the Wiener continued fraction attack
    '''
    frac = rational_to_contfrac(e, n)
    convergents = convergents_from_contfrac(frac)
    
    for (k,d) in convergents:
        
        #check if d is actually the key
        if k!=0 and (e*d-1)%k == 0:
            phi = (e*d-1)//k
            s = n - phi + 1
            # check if the equation x^2 - s*x + n = 0 has integer roots
            discr = s*s - 4*n
            if(discr>=0):
                t = is_perfect_square(discr)
                if t!=-1 and (s+t)%2==0:
                    print("Successfully hacked with Wiener attack!")
                    return d

In [ ]:
'''
Created on Dec 14, 2011
@author: pablocelayes
'''

#!/usr/bin/python
# -*- coding: utf-8 -*-
"""\
This module generates RSA-keys which are vulnerable to
the Wiener continued fraction attack
(see RSAfracCont.pdf)
The RSA keys are obtained as follows:
1. Choose two prime numbers p and q
2. Compute n=pq
3. Compute phi(n)=(p-1)(q-1)
4. Choose e coprime to phi(n) such that gcd(e,n)=1
5. Compute d = e^(-1) mod (phi(n))
6. e is the publickey; n is also made public (determines the block size); d is the privatekey
Encryption is as follows:
1. Size of data to be encrypted must be less than n
2. ciphertext=pow(plaintext,publickey,n)
Decryption is as follows:
1. Size of data to be decrypted must be less than n
2. plaintext=pow(ciphertext,privatekey,n)
-------------------------------
RSA-keys are Wiener-vulnerable if d < (n^(1/4))/sqrt(6)
"""

In [39]:
def test_hack_RSA():
    print("Testing Wiener Attack")
    times = 5
    
    while(times>0):
        e,n,d = generateKeys(1024)
        print("(e,n) is (", e, ", ", n, ")")
        print("d = ", d)
    
        hacked_d = hack_RSA(e, n)
    
        if d == hacked_d:
            print("Hack WORKED!")
        else:
            print("Hack FAILED")
        
        print("d = ", d, ", hacked_d = ", hacked_d)
        print("-------------------------")
        times -= 1

In [40]:
test_hack_RSA()

Testing Wiener Attack
(e,n) is ( 23212386923349889732950970438943004476807044515002255950906251322942193651138296200960186615227173687114819283470602328812549981232915400770350153177355225267464592992063753419741613464988190570150755997300402155204703121899375149449051218000942046889969609096151427992751145613079577041694167007578689571849 ,  379059366269973633023827533291422461291244224364587758590599563838657447189265245559723424565073103869615945830612567053320077536635140375344466823123429561688550974790675891063722717947003788560361030227865087085843684990666275790200425240027060650200438308023211970691135881768509470144878807202711697140753 )
d =  31166425529010883437887981752432122128872310147355145785619038844242340746041
Successfully hacked with Wiener attack!
Hack WORKED!
d =  31166425529010883437887981752432122128872310147355145785619038844242340746041 , hacked_d =  31166425529010883437887981752432122128872310147355145785619038844242340746041
-------------------------
(e,n) 